# Improved Glove Rnn

In [1]:
import os
# use cpu
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np, pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D, SimpleRNN, GRU
from keras.models import Model
# from keras import initializers, regularizers, constraints, optimizers, layers

2024-05-25 16:31:40.095093: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-25 16:31:40.213303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# quick and dirty way to change the current working directory to root (/toxic-comment-classification)
# you should run this at least once just to be certain
from os import chdir, path, getcwd
if getcwd().endswith("src"):
    chdir(path.pardir)
if path.isfile("checkcwd"):
    print("Success")
else:
    raise Exception("Something went wrong. cwd=" + getcwd())
root_path = os.getcwd()

Success


In [3]:
path = 'kaggle/input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
EMBEDDING_FILE=f'{path}glove_embeddings/glove.6B.300d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_vanilla2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_vanilla2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'

Embedding parameters

In [4]:
embed_size = 300 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

Read data

In [5]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read().splitlines()
    
list_sentences_train = read_from_file(CLEAN_TRAIN_DATA_FILE)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = read_from_file(CLEAN_TEST_DATA_FILE)

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [6]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [7]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [8]:
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(-0.0039050116, 0.38177028)

In [9]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Create Model

In [10]:
from keras.metrics import AUC

def get_model(layertype='RNN', use_dropout=False, dropout_rate=0.1):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    if layertype == 'RNN' and not use_dropout:
        x = Bidirectional(SimpleRNN(50, return_sequences=True))(x)
    elif layertype == 'LSTM' and not use_dropout:
        x = Bidirectional(LSTM(50, return_sequences=True))(x)
    elif layertype == 'GRU' and not use_dropout:
        x = Bidirectional(GRU(50, return_sequences=True))(x)
    elif layertype == 'RNN' and use_dropout:
        x = Bidirectional(SimpleRNN(50, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    elif layertype == 'LSTM' and use_dropout:
        x = Bidirectional(LSTM(50, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    elif layertype == 'GRU' and use_dropout:
        x = Bidirectional(GRU(50, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[AUC(name='auc')])
    return model

Create Validation Set

In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X_t, y, test_size=0.1, random_state=7)

Training and eval

In [12]:
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.metrics import roc_auc_score

checkpoint_path = 'model_checkpoint/'

class RocAucEvaluation(Callback):
    def __init__(self, checkpoint_file, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.max_score = 0
        self.not_better_count = 0
        self.checkpoint_file = checkpoint_file

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            if (score > self.max_score):
                print("*** New High Score (previous: %.6f) \n" % self.max_score)
                self.model.save(self.checkpoint_file)
                self.max_score = score
                self.not_better_count = 0
            else:
                self.not_better_count += 1
                if self.not_better_count > 3:
                    print("Epoch %05d: early stopping, high score = %.6f" % (epoch,self.max_score))
                    self.model.stop_training = True


# rnn_checkpoint = ModelCheckpoint(checkpoint_path + 'rnn.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
# lstm_checkpoint = ModelCheckpoint(checkpoint_path + 'lstm.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
# gru_checkpoint = ModelCheckpoint(checkpoint_path + 'gru.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)

# early_stopping = EarlyStopping(monitor='loss', min_delta=0.0005, restore_best_weights=True)

In [13]:
# import matplotlib.pyplot as plt

# def plot_loss(callback):
#     plt.figure(figsize=(12,6))
#     plt.plot(callback.losses)
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('500 batch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    
# def plot_auc(callback):
#     plt.figure(figsize=(12,6))
#     plt.plot(callback.aucs)
#     plt.title('model AUC')
#     plt.ylabel('AUC')
#     plt.xlabel('500 batch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

# def plot_loss_auc(history):
#     plot_loss(history)
#     plot_auc(history)

In [14]:
import gc
gc.collect()

0

In [15]:
# model = get_model('RNN', True, 0.15)
# model.summary()

In [16]:
batch_size = 64
epochs = 3
# model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[rnn_checkpoint], validation_split=0.1)

And finally, get predictions for the test set and prepare a submission CSV:

In [17]:
# model.load_weights(checkpoint_path + 'rnn.keras')
# sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
# y_test = model.predict([X_te], batch_size=1024, verbose=1)
# sample_submission[list_classes] = y_test
# sample_submission.to_csv(root_path + '/kaggle/working/' + 'rnn_glove_submission.csv', index=False)

In [18]:
from keras import backend as K
from sklearn.model_selection import KFold
import keras
keras.config.disable_traceback_filtering()

model = get_model('RNN', True, 0.15)

num_folds = 10

gc.collect()
K.clear_session()

# Uncomment for out-of-fold predictions
scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_t):
    
    kfold_y_train, kfold_y_test = y[train_index], y[test_index]
    kfold_X_train = X_t[train_index]

    kfold_X_valid = X_t[test_index]

    
    gc.collect()
    K.clear_session()
    
    model = get_model('RNN', True, 0.15)
    
    
    rnn_checkpoint = RocAucEvaluation(checkpoint_file=checkpoint_path + 'rnn.keras',
                                      validation_data=([kfold_X_valid], kfold_y_test))
    
    model.fit([kfold_X_train], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
             callbacks = [rnn_checkpoint])
    
    gc.collect()
    
    #model.load_weights(bst_model_path)
    model.load_weights(checkpoint_path + 'rnn.keras')
    
    predict += model.predict([X_te], batch_size=batch_size, verbose=1) / num_folds
    
    gc.collect()
    # uncomment for out of fold predictions
    oof_predict[test_index] = model.predict([kfold_X_valid], batch_size=batch_size, verbose=1)
    cv_score = roc_auc_score(kfold_y_test, oof_predict[test_index])
    
    scores.append(cv_score)
    print('score: ', cv_score)


print("Done")
#
print('Total CV score is {}'.format(np.mean(scores)))

sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
sample_submission[list_classes] = predict
sample_submission.to_csv(root_path + '/kaggle/working/' + '1_sub.csv', index=False)

# uncomment for out of fold predictions
oof = pd.DataFrame.from_dict({'id': train['id']})
for c in list_classes:
   oof[c] = np.zeros(len(train))
   
oof[list_classes] = oof_predict
oof.to_csv(root_path + '/kaggle/working/' + '1_oof.csv', index=False)

2024-05-25 15:33:57.378621: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-25 15:33:57.481107: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-25 15:33:57.481144: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-25 15:33:57.483924: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-25 15:33:57.483969: I external/local_xla/xla/stream_executor

Epoch 1/3


I0000 00:00:1716626041.718562  131699 service.cc:145] XLA service 0x7f0e8c024830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716626041.718653  131699 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-05-25 15:34:01.784579: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-25 15:34:02.035036: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   5/2244 ━━━━━━━━━━━━━━━━━━━━ 1:20 36ms/step - auc: 0.5684 - loss: 0.8090

I0000 00:00:1716626047.103764  131699 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step

 ROC-AUC - epoch: 1 - score: 0.977421 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 87s 36ms/step - auc: 0.9163 - loss: 0.0997
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

 ROC-AUC - epoch: 2 - score: 0.980941 

*** New High Score (previous: 0.977421) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 78s 35ms/step - auc: 0.9824 - loss: 0.0476
Epoch 3/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

 ROC-AUC - epoch: 3 - score: 0.982846 

*** New High Score (previous: 0.980941) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 75s 33ms/step - auc: 0.9876 - loss: 0.0426
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score:  0.9828456283259323
Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step

 ROC-AUC - epoch: 1 - score: 0.976976 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 87s 36ms/step - auc: 0.9198 - loss: 0.0936
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

 ROC-AUC - epo

Build Lstm model

In [18]:
# del model
# model = get_model('LSTM', True, 0.15)
# model.summary()

In [19]:
# model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[lstm_checkpoint], validation_split=0.1)

In [22]:
# model.load_weights(checkpoint_path + 'lstm.keras')
# sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
# y_test = model.predict([X_te], batch_size=1024, verbose=1)
# sample_submission[list_classes] = y_test
# sample_submission.to_csv(root_path + '/kaggle/working/' + 'lstm_glove_submission.csv', index=False)

In [23]:
from keras import backend as K
from sklearn.model_selection import KFold
import keras
keras.config.disable_traceback_filtering()

model = get_model('LSTM', True, 0.15)

num_folds = 10

gc.collect()
K.clear_session()

# Uncomment for out-of-fold predictions
scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_t):
    
    kfold_y_train, kfold_y_test = y[train_index], y[test_index]
    kfold_X_train = X_t[train_index]

    kfold_X_valid = X_t[test_index]

    
    gc.collect()
    K.clear_session()
    
    model = get_model('LSTM', True, 0.15)
    
    
    lstm_checkpoint = RocAucEvaluation(checkpoint_file=checkpoint_path + 'lstm.keras',
                                      validation_data=([kfold_X_valid], kfold_y_test))
    
    model.fit([kfold_X_train], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
             callbacks = [lstm_checkpoint])
    
    gc.collect()
    
    #model.load_weights(bst_model_path)
    model.load_weights(checkpoint_path + 'lstm.keras')
    
    predict += model.predict([X_te], batch_size=batch_size, verbose=1) / num_folds
    
    gc.collect()
    # uncomment for out of fold predictions
    oof_predict[test_index] = model.predict([kfold_X_valid], batch_size=batch_size, verbose=1)
    cv_score = roc_auc_score(kfold_y_test, oof_predict[test_index])
    
    scores.append(cv_score)
    print('score: ', cv_score)


print("Done")
#
print('Total CV score is {}'.format(np.mean(scores)))

sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
sample_submission[list_classes] = predict
sample_submission.to_csv(root_path + '/kaggle/working/' + '2_sub.csv', index=False)

# uncomment for out of fold predictions
oof = pd.DataFrame.from_dict({'id': train['id']})
for c in list_classes:
   oof[c] = np.zeros(len(train))
   
oof[list_classes] = oof_predict
oof.to_csv(root_path + '/kaggle/working/' + '2_oof.csv', index=False)

Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

 ROC-AUC - epoch: 1 - score: 0.984198 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 199s 87ms/step - auc: 0.9320 - loss: 0.0846
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

 ROC-AUC - epoch: 2 - score: 0.987039 

*** New High Score (previous: 0.984198) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 202s 90ms/step - auc: 0.9863 - loss: 0.0430
Epoch 3/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step

 ROC-AUC - epoch: 3 - score: 0.986695 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 197s 88ms/step - auc: 0.9898 - loss: 0.0373
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step
score:  0.9870386551191438
Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

 ROC-AUC - epoch: 1 - score: 0.981833 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 204s 90ms/step - auc: 0.9262 - loss: 0.0852
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

 ROC-AUC - epoch: 2 - score: 0.98352

Build GRU model

In [24]:
del model
# model = get_model('GRU', True, 0.1)
# model.summary()

In [25]:
# model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[gru_checkpoint], validation_split=0.1)

In [26]:
# model.load_weights(checkpoint_path + 'gru.keras')
# sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
# y_test = model.predict([X_te], batch_size=1024, verbose=1)
# sample_submission[list_classes] = y_test
# sample_submission.to_csv(root_path + '/kaggle/working/' + 'gru_glove_submission.csv', index=False)

In [27]:
model = get_model('GRU', True, 0.1)

num_folds = 10

gc.collect()
K.clear_session()

# Uncomment for out-of-fold predictions
scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_t):
    
    kfold_y_train, kfold_y_test = y[train_index], y[test_index]
    kfold_X_train = X_t[train_index]

    kfold_X_valid = X_t[test_index]

    
    gc.collect()
    K.clear_session()
    
    model = get_model('GRU', True, 0.1)
    
    
    gru_checkpoint = RocAucEvaluation(checkpoint_file=checkpoint_path + 'gru.keras',
                                      validation_data=([kfold_X_valid], kfold_y_test))
    
    model.fit([kfold_X_train], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
             callbacks = [gru_checkpoint])
    
    gc.collect()
    
    #model.load_weights(bst_model_path)
    model.load_weights(checkpoint_path + 'gru.keras')
    
    predict += model.predict([X_te], batch_size=batch_size, verbose=1) / num_folds
    
    gc.collect()
    # uncomment for out of fold predictions
    oof_predict[test_index] = model.predict([kfold_X_valid], batch_size=batch_size, verbose=1)
    cv_score = roc_auc_score(kfold_y_test, oof_predict[test_index])
    
    scores.append(cv_score)
    print('score: ', cv_score)


print("Done")
#
print('Total CV score is {}'.format(np.mean(scores)))

sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
sample_submission[list_classes] = predict
sample_submission.to_csv(root_path + '/kaggle/working/' + '3_sub.csv', index=False)

# uncomment for out of fold predictions
oof = pd.DataFrame.from_dict({'id': train['id']})
for c in list_classes:
   oof[c] = np.zeros(len(train))
   
oof[list_classes] = oof_predict
oof.to_csv(root_path + '/kaggle/working/' + '3_oof.csv', index=False)

Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step

 ROC-AUC - epoch: 1 - score: 0.979245 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 199s 87ms/step - auc: 0.9261 - loss: 0.0867
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step

 ROC-AUC - epoch: 2 - score: 0.982980 

*** New High Score (previous: 0.979245) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 191s 85ms/step - auc: 0.9846 - loss: 0.0544
Epoch 3/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step

 ROC-AUC - epoch: 3 - score: 0.983922 

*** New High Score (previous: 0.982980) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 191s 85ms/step - auc: 0.9884 - loss: 0.0482
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 36s 15ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
score:  0.983921909982631
Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step

 ROC-AUC - epoch: 1 - score: 0.977426 

*** New High Score (previous: 0.000000) 

2244/2244 ━━━━━━━━━━━━━━━━━━━━ 194s 85ms/step - auc: 0.9273 - loss: 0.0910
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/

In [28]:
# lstm_submission = pd.read_csv(root_path + '/kaggle/working/' + 'lstm_glove_submission.csv')
# vanilla_submission = pd.read_csv(root_path + '/kaggle/working/' + 'vanilla_submission.csv')
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# ensemble_submission = lstm_submission.copy()
# ensemble_submission[label_cols] = (vanilla_submission[label_cols] + lstm_submission[label_cols]) / 2
# ensemble_submission.to_csv(root_path + '/kaggle/working/' + 'nbsvm_lstm_ensemble_submission.csv', index=False)